In [3]:
import random

class NetworkACO:
    def __init__(self, graph, alpha=1.0, beta=2.0, evaporation_rate=0.5, Q=100):
        """
        graph: dict of node: {neighbor: weight}
        alpha: pheromone importance
        beta: heuristic importance (inverse of weight)
        evaporation_rate: pheromone evaporation
        Q: pheromone deposit factor
        """
        self.graph = graph
        self.nodes = list(graph.keys())
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.Q = Q

        # Initialize pheromone levels on edges
        self.pheromone = {}
        for node in graph:
            for neighbor in graph[node]:
                self.pheromone[(node, neighbor)] = 1.0  # initial pheromone

    def _heuristic(self, node1, node2):
        # Inverse of the edge weight (distance or cost)
        return 1.0 / self.graph[node1][node2]

    def _probabilities(self, current_node, visited):
        neighbors = self.graph[current_node].keys()
        probs = []
        total = 0.0

        for neighbor in neighbors:
            if neighbor not in visited:
                pheromone = self.pheromone[(current_node, neighbor)] ** self.alpha
                heuristic = self._heuristic(current_node, neighbor) ** self.beta
                prob = pheromone * heuristic
                probs.append((neighbor, prob))
                total += prob

        # Normalize probabilities
        if total == 0:
            return []
        return [(node, prob / total) for node, prob in probs]

    def _select_next_node(self, probabilities):
        r = random.random()
        cumulative = 0.0
        for node, prob in probabilities:
            cumulative += prob
            if r <= cumulative:
                return node
        return probabilities[-1][0]

    def find_path(self, start, end):
        path = [start]
        visited = set(path)

        while path[-1] != end:
            current = path[-1]
            probabilities = self._probabilities(current, visited)
            if not probabilities:
                # Dead end, no unvisited neighbors
                return None
            next_node = self._select_next_node(probabilities)
            path.append(next_node)
            visited.add(next_node)

        return path

    def _path_length(self, path):
        length = 0.0
        for i in range(len(path) - 1):
            length += self.graph[path[i]][path[i+1]]
        return length

    def update_pheromones(self, paths):
        # Evaporation
        for edge in self.pheromone:
            self.pheromone[edge] *= (1 - self.evaporation_rate)
            if self.pheromone[edge] < 0.1:
                self.pheromone[edge] = 0.1  # minimum pheromone

        # Deposit
        for path in paths:
            if path is None:
                continue
            length = self._path_length(path)
            deposit_amount = self.Q / length
            for i in range(len(path) - 1):
                edge = (path[i], path[i+1])
                self.pheromone[edge] += deposit_amount

def main():
    # Example network graph: node -> {neighbor: cost}
    graph = {
        'A': {'B': 1, 'C': 4},
        'B': {'A': 1, 'C': 2, 'D': 5},
        'C': {'A': 4, 'B': 2, 'D': 1},
        'D': {'B': 5, 'C': 1}
    }

    aco = NetworkACO(graph)

    start = 'A'
    end = 'D'
    iterations = 10
    ants_per_iteration = 5

    for i in range(iterations):
        paths = []
        for _ in range(ants_per_iteration):
            path = aco.find_path(start, end)
            if path:
                print(f"Iteration {i+1}, Ant path: {path}")
            paths.append(path)
        aco.update_pheromones(paths)

    # After iterations, find best path based on pheromones
    best_path = aco.find_path(start, end)
    print(f"Best path after {iterations} iterations: {best_path}")

if __name__ == "__main__":
    main()


Iteration 1, Ant path: ['A', 'B', 'C', 'D']
Iteration 1, Ant path: ['A', 'B', 'C', 'D']
Iteration 1, Ant path: ['A', 'B', 'C', 'D']
Iteration 1, Ant path: ['A', 'B', 'C', 'D']
Iteration 1, Ant path: ['A', 'C', 'D']
Iteration 2, Ant path: ['A', 'B', 'C', 'D']
Iteration 2, Ant path: ['A', 'B', 'C', 'D']
Iteration 2, Ant path: ['A', 'B', 'C', 'D']
Iteration 2, Ant path: ['A', 'B', 'C', 'D']
Iteration 2, Ant path: ['A', 'B', 'C', 'D']
Iteration 3, Ant path: ['A', 'B', 'C', 'D']
Iteration 3, Ant path: ['A', 'B', 'C', 'D']
Iteration 3, Ant path: ['A', 'B', 'C', 'D']
Iteration 3, Ant path: ['A', 'B', 'C', 'D']
Iteration 3, Ant path: ['A', 'B', 'C', 'D']
Iteration 4, Ant path: ['A', 'B', 'C', 'D']
Iteration 4, Ant path: ['A', 'B', 'C', 'D']
Iteration 4, Ant path: ['A', 'B', 'C', 'D']
Iteration 4, Ant path: ['A', 'B', 'C', 'D']
Iteration 4, Ant path: ['A', 'B', 'C', 'D']
Iteration 5, Ant path: ['A', 'B', 'C', 'D']
Iteration 5, Ant path: ['A', 'B', 'C', 'D']
Iteration 5, Ant path: ['A', 'B', 'C'